In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from fastai.tabular.all import *
import gradio as gr

In [2]:
path = Path()
df = pd.read_csv("rookie_year.csv")
learn = load_learner(path/"export.pkl")

In [64]:
def predict(data):
    columns = ["Name", "G", "GS", "Cmp", "Att", "Yds", "Cmp%", "TD", "Int", "Y/G", "Sk"]
    row = df[df["Name"] == data]
    row = row.loc[:, ~df.columns.str.contains('^Unnamed')]
    if not len(row):
        print("ERROR: No QB in database with this name")
        return        
    pred_row, clas, probs = learn.predict(row.iloc[0])
    prediction = pred_row.decode()["Tier"].item()    
    return row[columns], prediction


In [81]:
demo = gr.Interface(fn=predict, 
                    inputs="text", 
                    outputs=[
                        gr.Dataframe(row_count=1, col_count=11, headers=columns, label="Rookie Year Stats"), 
                        gr.Textbox(label="Prediction")
                    ],
                    title="Rookie QB Career Prediction (Name)",
                    description="Given Name of QB who has played in the NFL, predict their career tier. Uses data from https:\/\/www.pro-football-reference.com. Tiers based on PFR Approximate Value.",
                    examples=["Tom Brady", "Joe Burrow", "Trevor Lawrence"]
                   )

demo.launch()

Running on local URL:  http://127.0.0.1:7866/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x26e1100f1c0>, 'http://127.0.0.1:7866/', None)

In [72]:
def predict2(data):
    row = data.drop("Name", axis=1).astype(float)
    row["Cmp"] = row["Att"].item() * row["Cmp%"].item()
    pred_row, clas, probs = learn.predict(row.iloc[0])
    prediction = pred_row.decode()["Tier"].item()    
    return prediction


In [80]:
demo2 = gr.Interface(fn=predict2, 
                    inputs=gr.Dataframe(row_count=1, col_count=8, headers=[x for x in columns if x not in ["Cmp", "G", "GS"]], label="Rookie Year Stats"), 
                    outputs=gr.Textbox(label="Prediction"),
                    title="Rookie QB Career Prediction (Stats)",
                    description="Given stats of a presumed rookie QB, predict their career tier. Uses data from https:\/\/www.pro-football-reference.com. Tiers based on PFR Approximate Value."
                    )

demo2.launch()

Running on local URL:  http://127.0.0.1:7865/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x26e10daa310>, 'http://127.0.0.1:7865/', None)